# Exploratory Data Analysis - Insurance Policy Generation Chatbot

Exploratory Data Analysis (EDA) for textual data from PDFs is a multi-step process that involves:


## Setup & Initialization:

Let's Import dependencies and initialize some variables.


In [1]:
%pip install -r requirements.txt > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [39]:
%load_ext autoreload
%autoreload 2

import glob
import os

# import openai api and set api key
import openai

# import src modules
from src import config
from src import ETL

# import langchain related modules
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


openai.api_key = config.OPENAI_API_KEY

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Extraction:

In this section, we will download the data from an s3 bucket and save it locally.


In [3]:
ETL.extract_from_s3(
    config.S3_BUCKET_NAME,
    config.S3_BUCKET_PREFIX,
    config.AWS_ACCESS_KEY_ID,
    config.AWS_SECRET_ACCESS_KEY,
    config.DATASET_ROOT_PATH,
)

Files from anyoneai-datasets/queplan_insurance/ already downloaded to /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset.


## Document Loading:

Now let's load the PDFs containing Insurance Policies using PyPDFDirectoryLoader from LangChain.


In [4]:
loader = PyPDFDirectoryLoader(config.DATASET_ROOT_PATH)

documents = loader.load()

Each Page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`


In [5]:
print(f"Number of pages: {len(documents)}")

Number of pages: 267


Let's take a look at it's metadata. We can see that it contains the following fields:

- `source` : The source of the document
- `page` : The page number of the document

this information can be used to trace back the document to the source for debugging purposes.


In [6]:
print("The Page's File name is: ", documents[0].metadata["source"])

print("The Page number is: ", documents[0].metadata["page"])

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320130223.pdf
The Page number is:  0


We can also add custom metadata to the document. Let's add a `policy_title` field to the metadata. This will be useful for us later on.

For this, we need to reload the documents with the new metadata.


In [7]:
documents = ETL.load_documents_with_title(config.DATASET_ROOT_PATH)

In [8]:
print(f"Number of pages: {len(documents)}")

print("The Page's File name is: ", documents[50].metadata["source"])

print("The Page number is: ", documents[50].metadata["page"])

print("The Policy's Title is: ", documents[50].metadata["title"])

Number of pages: 267
The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320200214.pdf
The Page number is:  3
The Policy's Title is:  SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO


Now, let's look at a glimpse of the first page of the first document.


In [9]:
print(documents[0].page_content[0:500], "...")

SEGURO COLECTIVO COMPLEMENTARIO DE SALUD 
Incorporada al Depósito de Pólizas bajo el código POL320130223
ARTICULO 1°: REGLAS APLICABLES AL CONTRATO
 
 
 
 
 
 
 
Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o beneficia ...


The reason why we explore the first page of the first document is to get a sense of the structure of the document. This analisis will help us in the next step which is the Document Splitting phase, where we will split the document into chunks to feed our vector store. In order to ensure that each chunk has coherent, self-contained information, we need to define de appropriate chunk size.

Given the content of our dataset, the logical division would be to break down by articles and major subheadings since they seem to encapsulate a singular topic or concept.


## Preprocessing:

However, we see the presence of a lot of whitespace and empty lines across the dataset. This might interfere with the chunking process and produce poor results. Therefore, we will preprocess the data to remove the empty lines and whitespace.


In [10]:
# This function will clean the pages from extra whitespaces and newlines
ETL.preprocess(documents)

[Document(page_content='SEGURO COLECTIVO COMPLEMENTARIO DE SALUD \nIncorporada al Depósito de Pólizas bajo el código POL320130223\nARTICULO 1°: REGLAS APLICABLES AL CONTRATO\n\nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o beneficiario.\n\nARTÍCULO Nº 2: COBERTURA\n\nLa compañía de seguros bajo las condiciones y términos que más adelante se establecen, conviene en\nreembolsar o pagar al beneficiario, los gastos médicos razonables y acostumbrados en que haya incurrido\nefectivamente un asegurado, en complemento de lo que cubra el sistema de salud previsional o de bienestar\nu otro seguro o convenio, a consecuencia de una incapacidad cubierta.', metadata={'source': '/home/gio/ANYONEAI/InsurancePolicyChatbot

In [11]:
print(documents[0].page_content)

SEGURO COLECTIVO COMPLEMENTARIO DE SALUD 
Incorporada al Depósito de Pólizas bajo el código POL320130223
ARTICULO 1°: REGLAS APLICABLES AL CONTRATO

Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o beneficiario.

ARTÍCULO Nº 2: COBERTURA

La compañía de seguros bajo las condiciones y términos que más adelante se establecen, conviene en
reembolsar o pagar al beneficiario, los gastos médicos razonables y acostumbrados en que haya incurrido
efectivamente un asegurado, en complemento de lo que cubra el sistema de salud previsional o de bienestar
u otro seguro o convenio, a consecuencia de una incapacidad cubierta.


Now that we have preprocessed the data, we can proceed to build our LLM.


# LLM Pipeline

Now that we have our data loaded and preprocessed, we can start building our LLM pipeline.


## Document Splitting

Now, let's split the document into chunks of text for further processing.


### Chunking Considerations

Several variables play a role in determining the best chunking strategy, and these variables vary depending on the use case. Here are some key aspects to keep in mind:

1. **What is the nature of the content being indexed?**

   Based on the content of the dataset, we're working with insurance policies. The logical division would be to break down by articles and major subheadings since they seem to encapsulate a singular topic or concept.

2. **Which embedding model will be used, and what chunk sizes does it perform optimally on?**

   We will be using OpenAI GPT-3.5 Turbo, which performs optimally on chunks of 512 tokens.

3. **What are your expectations for the length and complexity of user queries?**

   Since this solution will act as a chatbot, we can expect the queries to be mostly short and simple. However, we should also consider the possibility of more complex queries, such as "Cual es la diferencia entre el seguro de vida y el seguro de salud?" (What is the difference between life insurance and health insurance?)

4. **How will the retrieved results be utilized within your specific application?**

   The retrieved results will be used to answer user queries. The user will be able to ask questions about the content of the documents, and the chatbot will respond with the most relevant information.


### Chunking methods

There are different methods for chunking, and each of them might be appropriate for different situations. By examining the strengths and weaknesses of each method, our goal is to identify the right scenario to apply them to.


#### Fixed-size chunking

This is the most common and straightforward approach to chunking: we simply decide the number of tokens in our chunk and, optionally, whether there should be any overlap between them.


In [12]:
c_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150, separator="\n")

c_splitter_result = c_splitter.split_documents(documents)

print("Chunk Size count: ", len(c_splitter_result))

print(c_splitter_result[0].page_content)
print("-------------------")
print(c_splitter_result[1].page_content)
print("-------------------")
print(c_splitter_result[2].page_content)

Chunk Size count:  724
SEGURO COLECTIVO COMPLEMENTARIO DE SALUD 
Incorporada al Depósito de Pólizas bajo el código POL320130223
ARTICULO 1°: REGLAS APLICABLES AL CONTRATO
Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o beneficiario.
ARTÍCULO Nº 2: COBERTURA
La compañía de seguros bajo las condiciones y términos que más adelante se establecen, conviene en
reembolsar o pagar al beneficiario, los gastos médicos razonables y acostumbrados en que haya incurrido
efectivamente un asegurado, en complemento de lo que cubra el sistema de salud previsional o de bienestar
u otro seguro o convenio, a consecuencia de una incapacidad cubierta.
-------------------
Se otorgará cobertura a los gastos médicos incurridos por los 

#### Recursive Chunking

Recursive chunking divides the input text into smaller chunks in a hierarchical and iterative manner using a set of separators. If the initial attempt at splitting the text doesn’t produce chunks of the desired size or structure, the method recursively calls itself on the resulting chunks with a different separator or criterion until the desired chunk size or structure is achieved. This means that while the chunks aren’t going to be exactly the same size, they’ll still “aspire” to be of a similar size.


In [13]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    strip_whitespace=True,
)

r_splitter_result = r_splitter.split_documents(documents)

print("Chunk Size count: ", len(r_splitter_result))

print(r_splitter_result[0].page_content)
print("-------------------")
print(r_splitter_result[1].page_content)
print("-------------------")
print(r_splitter_result[2].page_content)

Chunk Size count:  770
SEGURO COLECTIVO COMPLEMENTARIO DE SALUD 
Incorporada al Depósito de Pólizas bajo el código POL320130223
ARTICULO 1°: REGLAS APLICABLES AL CONTRATO

Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o beneficiario.

ARTÍCULO Nº 2: COBERTURA

La compañía de seguros bajo las condiciones y términos que más adelante se establecen, conviene en
reembolsar o pagar al beneficiario, los gastos médicos razonables y acostumbrados en que haya incurrido
efectivamente un asegurado, en complemento de lo que cubra el sistema de salud previsional o de bienestar
u otro seguro o convenio, a consecuencia de una incapacidad cubierta.
-------------------
Se otorgará cobertura a los gastos médicos incurridos por l

## Storage

Now that we have our chunks, we can feed them to our vector store. But first, we need to convert them into embeddings.

### Embedding

We will use the OpenAI GPT-3.5 Turbo model to generate embeddings for our chunks.


In [14]:
embedding = OpenAIEmbeddings()

### Vector Store

We will use Chroma to store our embeddings. Chroma is a vector store that allows us to store and query embeddings.


In [15]:
if not glob.glob(os.path.join(config.CHROMA_PATH, "*.sqlite3")):
    print("Creating new Chroma DB")
    vector_store = Chroma.from_documents(
        r_splitter_result, embedding, persist_directory=config.CHROMA_PATH
    )
else:
    print("Loading Chroma DB")
    vector_store = Chroma(
        persist_directory=config.CHROMA_PATH, embedding_function=embedding
    )

Loading Chroma DB


Let's see the vector count.


In [16]:
print(vector_store._collection.count())

770


### Testing Similarity Search


Now that we have our vector store, let's test it by performing a similarity search on a query.


In [17]:
question = "Cuales son algunas de las limitaciones de la cobertura?"

Here, we retrieve the top 2 most similar chunks to our query.


In [18]:
response_ss = vector_store.similarity_search(question, k=2)

Let's see how many chunks we have retrieved.


In [19]:
len(response_ss)

2

Now, we will print the top 2 most similar chunks to our query.


In [20]:
for page in response_ss:
    print("The Page's File name is: ", page.metadata["source"])
    print("The Page number is: ", page.metadata["page"])
    print(page.page_content)
    print("-------------------")

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320190074.pdf
The Page number is:  29
LIMITACIONES DE LAS COBERTURAS:

Sin perjuicio de los porcentajes y límites de reembolso o pago que puedan establecerse en las condiciones
particulares, la presente póliza contempla las siguientes limitaciones de cobertura:

1. En aquellos casos en que el asegurado no esté afiliado a un sistema de salud previsional, privado o
estatal, se considerará como gasto efectivamente incurrido el monto que resulte de la aplicación del
porcentaje que se señala en las Condiciones Particulares de la póliza sobre el gasto médico reclamado.
Sobre el monto resultante se aplicarán los porcentajes y límites de reembolso o pago definidos para cada
cobertura en el Cuadro de Beneficios de las Condiciones Particulares de la póliza.
-------------------
The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320130223.pdf
The Page number is:  14
LIMITACIONES DE LAS COBERT

Let's persist the vector store so we can use it later.


In [21]:
vector_store.persist()

### Failure modes

As you've seen in the previous section, the results are not perfect. Notice that we're getting duplicate information comming from different documents. We will start applying some retrieval strategies to improve the results.


## Retrieval

This section is the centerpiece of our Retrieval Augmented Generation (RAG) Flow and it's where we will apply different strategies to improve the results of our similarity search.


### Addressing Diversity: Maximum marginal relevance

This would be our first attempt to improve the diversity in the search results. `Maximum marginal relevance` (MMR) strives to achieve both relevance to the query and diversity among the results.


In [22]:
response_mmr = vector_store.max_marginal_relevance_search(
    question, k=2, fetch_k=3, lambda_=0.5
)

In [23]:
for page in response_mmr:
    print("The Page's File name is: ", page.metadata["source"])
    print("The Page number is: ", page.metadata["page"])
    print("The Policy's Title is: ", page.metadata["title"])
    print("")
    print(page.page_content)
    print("-------------------")

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320190074.pdf
The Page number is:  29
The Policy's Title is:  SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO

LIMITACIONES DE LAS COBERTURAS:

Sin perjuicio de los porcentajes y límites de reembolso o pago que puedan establecerse en las condiciones
particulares, la presente póliza contempla las siguientes limitaciones de cobertura:

1. En aquellos casos en que el asegurado no esté afiliado a un sistema de salud previsional, privado o
estatal, se considerará como gasto efectivamente incurrido el monto que resulte de la aplicación del
porcentaje que se señala en las Condiciones Particulares de la póliza sobre el gasto médico reclamado.
Sobre el monto resultante se aplicarán los porcentajes y límites de reembolso o pago definidos para cada
cobertura en el Cuadro de Beneficios de las Condiciones Particulares de la póliza.
-------------------
The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dat

### Addressing Specificity: working with metadata using self-query retrievers


What if there is a question related to a specific document? For example, "Cual es la cobertura en SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO?".

Fortunatelly, crhomaDb supports operations on `metadata`.

`metadata` provides context for each embedded chunk.

However, there is an interesting challenge here: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:

1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.


In [24]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        type="string",
        description="el nombre de archivo y codigo de la poliza de donde vino este fragmento, el formato es POL{codigo de poliza}.pdf",
    ),
    AttributeInfo(
        name="page", type="integer", description="El numero de pagina de la poliza"
    ),
    AttributeInfo(name="title", type="string", description="El titulo de la poliza"),
]

In [49]:
document_content_description = "Polizas de Seguro"
llm = OpenAI(temperature=0)
retriever_sqr = SelfQueryRetriever.from_llm(
    llm, vector_store, document_content_description, metadata_field_info, verbose=True
)

In [26]:
question = "Cual es la cobertura en la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"

In [27]:
retriever_results = retriever.get_relevant_documents(question)

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='cobertura poliza accidentes personales reembolso gastos medicos' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='title', value='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS') limit=None


In [28]:
for k in retriever_results:
    print("The Page's File name is: ", k.metadata["source"])
    print("The Page number is: ", k.metadata["page"])
    print("The Policy's Title is: ", k.metadata["title"])
    print("")
    print(k.page_content)

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL120190177.pdf
The Page number is:  0
The Policy's Title is:  PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS

PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS
Incorporada al Depósito de Pólizas bajo el código POL120190177
ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO
Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o el beneficiario.
ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA
La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los
Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y
pagado la cobertura del sistema de salud previsional

Now we can see that the results are more specific to the query. However, this can be improved even further.

### Last Trick: Compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. In this particular case, we will combine the `ContextualCompressionRetriever` with the `MMR` retriever to improve the quality of our results.


In [34]:
compressor = LLMChainExtractor.from_llm(llm)

In [35]:
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever(search_type="mmr")
)

In [36]:
question = "de que habla el Mantenimiento artificial de la vida?"

In [37]:
compressed_documents = compressor_retriever.get_relevant_documents(question)
ETL.pretty_print_docs(compressed_documents)

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

"MANTENIMIENTO ARTIFICIAL DE LA VIDA: tratamiento médico que sustituye o mantiene activas las funciones vitales del cuerpo, que temporal o permanentemente no pueden realizarse de forma independiente, e incluye la respiración asistida, la nutrición e hidratación artificiales y la reanimación cardiopulmonar."
----------------------------------------------------------------------------------------------------
Document 2:

"Mantenimiento artificial de la vida: el Asegurador no pagará el mantenimiento artificial de la vida cuando el paciente sufra de una lesión, enfermedad o padecimiento que requiera tratamiento para el mantenimiento artificial de la vida, cuando no se espere que dichos tratamientos resulten en la recuperación del asegurado"
----------------------------------------------------------------------------------------------------
Document 3:

Medicamentos Ambulatorios Inmunosupresores o Inmunomoduladores


Now, the documents are much more short and specific to the query.

## Question Answering

Now that we have our retriever, we can use it to answer questions. In this section, we will explore 3 different approaches such as **Map Reduce, Refine, and Map ReRank**

This is the template we will use for our question answering pipeline:

```python

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Utiliza las siguientes piezas de contexto para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡gracias por preguntar!" al final de la respuesta!
{contexto}
Pregunta: {pregunta}
Respuesta útil:"""

```

Let's set up our LLM Agent. We will use ``gp3-5-turbo`` for this task.

In [42]:
llm = ChatOpenAI(model_name=config.OPENAI_NAME, temperature=0.9)

In [43]:
question = "Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"

In [44]:
template_prompt = """
Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Utiliza las siguientes piezas de contexto para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡gracias por preguntar!" al final de la respuesta!
{contexto}
Pregunta: {pregunta}
Respuesta útil:"""

qa_chain_prompt = PromptTemplate.from_template(template_prompt)


Let's try the Retrieval QA with chain type ``map_reduce``.

In [50]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_sqr,
    chain_type="map_reduce"
)

In [51]:
results = qa_chain_mr({"query": question})

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='limitaciones cobertura poliza' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='title', value='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS') limit=None


In [52]:
print(results["result"])

 Las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS incluyen: el pago al prestador o reembolso al asegurado tendrá como límite el monto definido en el Arancel del Prestador, la cobertura solamente mientras el asegurado se encuentre dentro del territorio nacional, lesiones o dolencias o situación de salud preexistentes, tratamientos médicos quirúrgicos distintos de los necesarios a consecuencia de lesiones cubiertas por esta póliza, y los gastos médicos que no tengan como causa un Evento o que provengan o se originen por, o sean consecuencia de, o correspondan a complicaciones de.
